In [10]:
import pandas as pd
import numpy as np

# Load the matrix from the Excel file located in the 'data' folder
file_path = 'data/pixeles.xlsx'

matrix_data = pd.read_excel(file_path, sheet_name=0, header=None)

matrix = matrix_data.values
matrix

array([[0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [11]:
kernel_horizontal = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
kernel_vertical = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])

In [12]:
kernel_horizontal = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
kernel_vertical = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])

def apply_convolution(matrix, kernel):
    matrix_height, matrix_width = matrix.shape
    kernel_height, kernel_width = kernel.shape
    
    output_matrix = np.zeros((matrix_height - 2, matrix_width - 2))
    
    for i in range(matrix_height - 2):
        for j in range(matrix_width - 2):
            region = matrix[i:i+kernel_height, j:j+kernel_width]
            output_matrix[i, j] = np.sum(region * kernel)
    
    return output_matrix

conv_horizontal = apply_convolution(matrix, kernel_horizontal)
conv_vertical = apply_convolution(matrix, kernel_vertical)

print("Deteccion de bordes Horizontales:\n", conv_horizontal)
print("\nDeteccion de bordes Verticales:\n", conv_vertical)

Deteccion de bordes Horizontales:
 [[ 0.  0.  0.  0.]
 [-1. -1.  0.  0.]
 [ 1.  2.  1.  1.]
 [-1. -1. -1.  0.]]

Deteccion de bordes Verticales:
 [[ 3. -1. -3.  0.]
 [ 3. -1. -3.  0.]
 [ 3.  0. -3. -1.]
 [ 2.  0. -2. -1.]]


In [13]:
def apply_padding(matrix, padding_size):
    original_height, original_width = matrix.shape
    
    padded_matrix = np.zeros((original_height + 2 * padding_size, original_width + 2 * padding_size))
    
    padded_matrix[padding_size:padding_size + original_height, padding_size:padding_size + original_width] = matrix
    
    return padded_matrix

padding_size = 2
padded_matrix = apply_padding(matrix, padding_size)

print("Matriz Original:\n", matrix)
print("\nPadded Matriz:\n", padded_matrix)


Matriz Original:
 [[0 0 1 0 0 0]
 [0 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 1 1 0 0]
 [0 0 0 0 0 0]]

Padded Matriz:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [14]:
def apply_stride(matrix, kernel, stride):

    matrix_height, matrix_width = matrix.shape
    kernel_height, kernel_width = kernel.shape
    
    output_height = (matrix_height - kernel_height) // stride + 1
    output_width = (matrix_width - kernel_width) // stride + 1
    
    output_matrix = np.zeros((output_height, output_width))
    
    for i in range(0, matrix_height - kernel_height + 1, stride):
        for j in range(0, matrix_width - kernel_width + 1, stride):
            region = matrix[i:i+kernel_height, j:j+kernel_width]
            output_matrix[i // stride, j // stride] = np.sum(region * kernel)
    
    return output_matrix

stride_size = 2
kernel_type = "horizontal"

kernel_horizontal = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
kernel_vertical = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])

if kernel_type == "horizontal":
    kernel = kernel_horizontal
else:
    kernel = kernel_vertical

strided_matrix = apply_stride(matrix, kernel, stride_size)

print("Matriz Original:\n", matrix)
print("\nStrided Matriz (Stride={}):\n".format(stride_size), strided_matrix)


Matriz Original:
 [[0 0 1 0 0 0]
 [0 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 1 1 0 0]
 [0 0 0 0 0 0]]

Strided Matriz (Stride=2):
 [[0. 0.]
 [1. 1.]]


In [15]:
def generate_kernels(n):
    kernels = []
    for _ in range(n):
        kernel = np.random.randint(-1, 2, (3, 3))
        kernels.append(kernel)
    return kernels

def apply_stacking(matrix, n):
    kernels = generate_kernels(n)
    
    feature_maps = []
    for kernel in kernels:
        feature_map = apply_convolution(matrix, kernel)
        feature_maps.append(feature_map)
    
    return feature_maps

n_maps = 3
feature_maps = apply_stacking(matrix, n_maps)

for i, fmap in enumerate(feature_maps):
    print(f"Mapa {i+1}:\n", fmap)
    print("\n")


Mapa 1:
 [[-4.  0.  2.  0.]
 [-2.  0.  2.  0.]
 [-4. -2.  1.  0.]
 [-3.  0.  1.  1.]]


Mapa 2:
 [[2. 3. 1. 0.]
 [2. 2. 1. 0.]
 [1. 3. 1. 0.]
 [1. 2. 2. 1.]]


Mapa 3:
 [[ 0.  1. -1.  0.]
 [ 0.  1. -1.  0.]
 [-2. -1. -2. -1.]
 [ 1.  3.  1.  0.]]




In [16]:
def max_pooling(matrix, stride):
    matrix_height, matrix_width = matrix.shape
    
    output_height = (matrix_height - 2) // stride + 1
    output_width = (matrix_width - 2) // stride + 1
    
    pooled_matrix = np.zeros((output_height, output_width))
    
    for i in range(0, matrix_height - 1, stride):
        for j in range(0, matrix_width - 1, stride):
            region = matrix[i:i+2, j:j+2]
            pooled_matrix[i // stride, j // stride] = np.max(region)
    
    return pooled_matrix

stride_size = 2 
pooled_matrix = max_pooling(matrix, stride_size)

print("Matriz Original:\n", matrix)
print("\nPooled Matria (Max Pooling, Stride={}):\n".format(stride_size), pooled_matrix)

Original Matrix:
 [[0 0 1 0 0 0]
 [0 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 1 1 0 0]
 [0 0 0 0 0 0]]

Pooled Matrix (Max Pooling, Stride=2):
 [[1. 1. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
